In [188]:
# 라이브러리
import requests, re
import xmltodict
from urllib.parse import urlparse

In [189]:
key = 'kNSQvU5WeosgTXwCx1mTthdz93%2BlLXHKA7ZtzbuNArBuUVVP4akW5xsfp6R5JYuMH106DwcuJRTqXJHI4q%2BNjA%3D%3D'

# 사용자가 위치한 정류장 인식
def position(x, y, r):
    url = f'http://ws.bus.go.kr/api/rest/stationinfo/getStationByPos?ServiceKey={key}&tmX={x}&tmY={y}&radius={r}' # 서울특별시_정류소정보조회 서비스 中 7_getStaionsByPosList
    content = requests.get(url).content
    dict = xmltodict.parse(content)

    target_stationName = str(dict['ServiceResult']['msgBody']['itemList'][0]['stationNm'])
    target_arsId = str(dict['ServiceResult']['msgBody']['itemList'][0]['arsId'])
    msgStation = "현재 인식된 정류장은 " + target_stationName + "입니다."

    return target_stationName, target_arsId, msgStation

#=========================
target_stationName, target_arsId, msgStation = position(126.9233020726, 37.5563975089, 100)
print(target_stationName)
print(target_arsId)
print(msgStation)

홍대입구역
14015
현재 인식된 정류장은 홍대입구역입니다.


In [190]:
def allBusnum(target_arsId):
    busList = []
    url = f'http://ws.bus.go.kr/api/rest/stationinfo/getStationByUid?ServiceKey={key}&arsId={target_arsId}'
    content = requests.get(url).content
    dict = xmltodict.parse(content)

    for i in range(0, len(dict['ServiceResult']['msgBody']['itemList'])):
        busList.append(dict['ServiceResult']['msgBody']['itemList'][i]['busRouteId'])

    return busList # 특정 정류장에 오는 모든 버스

#=========================
busList = allBusnum(target_arsId)
print(busList)

['113900015', '100100047', '100100049', '100100087', '100100088', '100100089', '111000014', '100100586', '100100588', '100100286', '100100288', '100100330', '100100451', '100100447', '100100346', '111000010', '100100363', '100100552', '165000145', '165000146', '165000147', '165000148', '165000149', '165000150', '165000421', '165000154', '165000409', '165000381', '232000073', '232000061', '229000255', '218000116', '218000108', '219000015', '232000090', '232000071', '218000105', '232000105']


In [192]:
arrivalStation = '당산역' # 임시

In [193]:
def theBusnum(arrivalStation, busList):   
    thebuslist = []
    
    for i in range(0, len(busList)):
        url = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey={key}&busRouteId={busList[i]}'
        content = requests.get(url).content
        dict = xmltodict.parse(content)
        
        buslinelist = []  # 특정 버스의 노선 목록   
        for i in range(0, len(dict['ServiceResult']['msgBody']['itemList'])):
            buslinelist.append(dict['ServiceResult']['msgBody']['itemList'][i]['stationNm'])
         
        if arrivalStation in buslinelist:
            thebuslist.append(dict['ServiceResult']['msgBody']['itemList'][i]['busRouteId'])

    return thebuslist # arrivalStation 경유하는 버스 ID
#=========================
thebuslist = theBusnum(arrivalStation, busList)
print(thebuslist)

['100100088', '111000014', '100100586', '100100288']


In [194]:
def waiting(target_arsId, thebuslist):
    url = f'http://ws.bus.go.kr/api/rest/stationinfo/getStationByUid?ServiceKey={key}&arsId={target_arsId}'
    content = requests.get(url).content
    dict = xmltodict.parse(content)

    waitingBusnum = []
    waitingTime = []
    for i in range(0, len(dict['ServiceResult']['msgBody']['itemList'])):
        if dict['ServiceResult']['msgBody']['itemList'][i]['busRouteId'] in thebuslist:
            x = dict['ServiceResult']['msgBody']['itemList'][i]['rtNm']
            y = dict['ServiceResult']['msgBody']['itemList'][i]['arrmsg1']
            waitingBusnum.append(x)
            waitingTime.append(y)
    return(waitingBusnum, waitingTime) # 버스 번호, 남은 시간

#=========================
waitingBusnum, waitingTime = waiting(target_arsId, thebuslist)
print(waitingBusnum)
print(waitingTime)

['603', '761', 'N26', '5714']
['6분4초후[4번째 전]', '13분38초후[8번째 전]', '운행종료', '2분8초후[1번째 전]']


In [195]:
def waitdeep(waitingTime):
    x = []
    for i in waitingTime:
        if i == '곧 도착':
            i = 0
            x.append(i)
        elif i == '운행종료':
            i = 9999
            x.append(i)
        else:
            i = i.split('[')[0]
            i = re.findall('\d', i)
            i = ''.join(i)
            x.append(int(i))      
    return(waitingBusnum, waitingTime, x)

#=========================
waitingBusnum, waitingTime, x = waitdeep(waitingTime)
#print(waitingBusnum)
#print(waitingTime)
#print(x)
#print(x.index(min(x))

#최종 값
print(waitingBusnum[(x.index(min(x)))], waitingTime[(x.index(min(x)))])

5714 2분8초후[1번째 전]
